## **Исследование надежности заемщиков**

### **Описание проекта**

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

##### **1. Открываем таблицу и изучаем общую информацию о данных**

Импортируем библиотеку pandas и считываем данные из csv-файла. Сохраняем датафрейм в переменную data

In [1]:
import pandas as pd

In [2]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv') 

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выводим общую информацию о датафрейме

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


##### **2. Предобработка данных**

**2.1. Удаление пропусков**

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них - total_income - хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца income_type.

In [6]:
for i_type in data['income_type'].unique():
	data.loc[(data['income_type'] == i_type) & (data['total_income'].isna()), 'total_income'] = \
	data.loc[(data['income_type'] == i_type), 'total_income'].median()

**2.2 Обработка аномальных значений**

In [7]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выводим медианное значение трудового стажа days_employed в днях

In [8]:
data.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получаются аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится для исследования.

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [10]:
data = data.loc[(data['children'] != -1) & (data['children'] != 20)]

В столбце children есть два аномальных значения. Удаляем строки, в которых встречаются такие аномальные значения из датафрейма data.

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

**2.3  Удаление пропусков (продолжение)**

Заполняем пропуски в столбце days_employed медианными значениями по каждому типу занятости income_type.

In [12]:
for i_type in data['income_type'].unique():
	data.loc[(data['income_type'] == i_type) & (data['days_employed'].isna()), 'days_employed'] = \
	data.loc[(data['income_type'] == i_type), 'days_employed'].median()

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**2.4  Изменение типов данных**

Заменяем вещественный тип данных в столбце total_income на целочисленный с помощью метода astype().

In [14]:
data['total_income'] = data['total_income'].astype('int')

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     21402 non-null  float64
 2   dob_years         21402 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      21402 non-null  int64  
 11  purpose           21402 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


**2.5  Обработка дубликатов**

Обрабатываем неявные дубликаты в столбце education. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приводим их к нижнему регистру.

In [16]:
data['education'] = data['education'].str.lower()

In [17]:
data.duplicated().sum()

71

In [18]:
data = data.drop_duplicates()

**2.6  Категоризация данных**

На основании диапазонов, указанных ниже, создаем в датафрейме data столбец total_income_category с категориями:

0–30000 — 'E';

30001–50000 — 'D';

50001–200000 — 'C';

200001–1000000 — 'B';

1000001 и выше — 'A'.

Например, кредитополучателю с доходом 25000 нужно назначить категорию 'E', а клиенту, получающему 235000, — 'B'. Используйте собственную функцию с именем categorize_income() и метод apply().

In [19]:
def categorize_income(row):
	income = row['total_income']

	if income <= 30000:
		return 'E'
	if income <= 50000:
		return 'D'
	if income <= 200000:
		return 'C'
	if income <= 1000000:
		return 'B'
	return 'A'


In [20]:
data['total_income_category'] = data.apply(categorize_income, axis=1)

In [21]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создаем функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:

'операции с автомобилем',

'операции с недвижимостью',

'проведение свадьбы',

'получение образования'.

Например, если в столбце purpose находится подстрока 'на покупку автомобиля', то в столбце purpose_category должна появиться строка 'операции с автомобилем'.

In [22]:
def categorize_purpose(row):
	purpose = row['purpose']

	if 'автомобил' in purpose:
		return 'операции с автомобилем'
	if 'жиль' in purpose or 'недвижимост' in purpose:
		return 'операции с недвижимостью'
	if 'свадьб' in purpose:
		return 'проведение свадьбы'
	if 'образовани' in purpose:
		return 'получение образования'

In [23]:
data['purpose_category'] = data.apply(categorize_purpose, axis=1)

In [24]:
data['purpose_category'].unique()

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

### Шаг 3. Исследование данных

##### Задание 19. Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [25]:
def debt_ratio(data, index):
	pivot = data.pivot_table(index=index, values='debt', aggfunc=['count', 'sum', 'mean']).reset_index()
	pivot.columns = [index, 'Всего клиентов', 'Всего должников', 'Доля должников']
	display(pivot.sort_values(by='Доля должников', ascending=False))

In [26]:
debt_ratio(data, 'children')

,children,Всего клиентов,Всего должников,Доля должников
4,4,41,4,0.097561
2,2,2052,194,0.094542
1,1,4808,444,0.092346
3,3,330,27,0.081818
0,0,14091,1063,0.075438
5,5,9,0,0.000000


Если исключить категорию с 5 детьми ввиду маленькой выборки, то очевиден рост количества долгов по кредиту среди клиентов, у которых есть дети. Если у бездетных это 7.5%, то в дальнейшем варьируется между 8.1% и 9.75%. 

Показатель долгов растет пропорционально количеству детей, но на 3 резко снижается. Возможно, так же связано с маленькой выборкой.

##### Задание 20. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
debt_ratio(data, 'family_status')

,family_status,Всего клиентов,Всего должников,Доля должников
0,Не женат / не замужем,2796,273,0.097639
3,гражданский брак,4134,385,0.093130
4,женат / замужем,12261,927,0.075606
1,в разводе,1189,84,0.070648
2,вдовец / вдова,951,63,0.066246


Люди с семейным положением "Не женат / не замужем" и "гражданский брак" в большей степени подвержены невозврату кредита в срок

В каждой группе выборка достаточно большая

##### Задание 21. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [28]:
def debt_ratio(data, index):
	pivot = data.pivot_table(index=index, values='debt', aggfunc=['count', 'sum', 'mean']).reset_index()
	

In [29]:
debt_ratio(data, 'total_income_category')

Выборка по клиентам с самым высоким и самым низким доходам очень маленькая. Вероятно, людей из группы А в принципе меньше остальных, т.к. их доход более миллиона в месяц. Группа Е малочисленна, вероятно, потому, что этим людям чаще отказывали в кредите. Тем не менее, эта группа остается наиболее рискованной, даже после серьезного отсеивания системой банка.

Категории B и C являются самыми крупными по количеству клиентов. При этом B один из самых низких по доле задолжников, а C - один из самых высоких и превосходит B почти на 1.5%.

##### Задание 22. Как разные цели кредита влияют на его возврат в срок?

In [30]:
debt_ratio(data, 'purpose_category')

Наиболее проблемные с точки зрения возврата в срок кредиты на автомобиль и образование. Наименее - операции с недвижимостью и проведение свадьбы

##### Задание 23. Приведите возможные причины появления пропусков в исходных данных.

Глобально, причина возникновения пропусков может быть и технической, и вызванной человеческим фактором.

Вероятно, нужно провести анализ того, как проводится заполнение формы. Скорее всего, не заполнить поля days_employed и total_income сотрудник не мог, на этапе оформления кредита, т.к. чаще всего, такие поля являются обязательными для заполнения. Если это так, то, скорее всего, причина пропусков в этих столбцах - техническая.

##### Задание 24. Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

В нашем случае, мы заполняем медианным значением данные о доходах. Главное преимущество по отношению к среднему арифметическому в том, что мы не берем в расчет выбросы. Если у 95% людей доход будет в диапазоне 50.000-100.000, а у оставшихся 5% - 1.000.000, то среднее арифметическое будет неадекватно смещено вверх и не будет отражать средний доход.

### Шаг 4. Напишите общий вывод

Проведенный анализ дает ответы на поставленные вопросы.

Наличие детей увеличивает вероятность невозврата кредита в срок, однако их количество существенно не влияет на этот показатель.

Люди, не вступавшие в официальный брак чаще не возвращают кредит в срок, по сравнению с людьми в браке, разведенными или вдовствующими.

Пропорциональной зависимости между уровнем дохода и возвратом кредита в срок не обнаружено.

Кредиты на операции с недвижимостью и свадьбу реже всего возвращаются с нарушениями. Чаще всего, в срок не возвращают кредиты на операции с автомобилем и получение образования.

